# 1. Import Macroencuestas 2011, 2015 and 2019

In [ ]:
import os

import pandas as pd
import pyreadstat
from dotenv import load_dotenv

In [ ]:
load_dotenv()
root_path = os.getenv("ROOT_PATH")
datasets_dir = os.path.join(root_path, "datasets", "raw", "CIS")

path_2019 = os.path.join(root_path, datasets_dir, "CIS001-Macroencuesta2019", "3235.sav")
path_2015 = os.path.join(root_path, datasets_dir, "CIS002-Macroencuesta2015", "3027.sav")
path_2011 = os.path.join(root_path, datasets_dir, "CIS003-Macroencuesta2011", "2858.sav")

df_2019, meta = pyreadstat.read_sav(path_2019, apply_value_formats=True)
df_2015, meta = pyreadstat.read_sav(path_2015, apply_value_formats=True)
df_2011, meta = pyreadstat.read_sav(path_2011, apply_value_formats=True)

# 2. Extract target variables

In [ ]:
# Extract common variables and add metadata for both years
df_2019_subset = df_2019[["CUES", "CCAA", "PROV"]].copy()
df_2019_subset["ESTUDIO"] = 3235
df_2019_subset["AÑO"] = 2019
df_2019_subset["MES"] = "Septiembre"

df_2015_subset = df_2015[["CUES", "CCAA", "PROV"]].copy()
df_2015_subset["ESTUDIO"] = 3027
df_2015_subset["AÑO"] = 2014
df_2015_subset["MES"] = "Septiembre"

In [ ]:
# Define the variable mappings
vs_mapping_2019 = {
    "VS1P": ["M1P5_0_4", "M2P5_0_4"],
    "VS2P": ["M1P5_0_2", "M2P5_0_2"],
    "VS3P": ["M1P5_0_8", "M2P5_0_8"],
    "VS1FP": ["M3P2_4"],
    "VF1P": ["M1P4_0_1", "M2P4_0_1"],
    "VF2P": ["M1P4_0_2", "M2P4_0_2"],
    "VF3P": ["M1P4_0_3", "M2P4_0_3"],
    "VF4P": ["M1P4_0_4", "M2P4_0_4"],
    "VF5P": ["M1P4_0_5", "M2P4_0_5"],
    "VF6P": ["M1P4_0_6", "M2P4_0_6"],
    "VF2FP": ["M3P1_2"],
    "VF5FP": ["M3P1_5"],
    "VF6FP": ["M3P1_6"],
}

vs_mapping_2015 = {
    "VS1P": ["P2201", "P3101"],
    "VS2P": ["P2202", "P3102"],
    "VS3P": ["P2203", "P3103"],
    "VS1FP": ["P52"],
    "VF1P": ["P2101", "P3001"],
    "VF2P": ["P2102", "P3002"],
    "VF3P": ["P2103", "P3003"],
    "VF4P": ["P2104", "P3004"],
    "VF5P": ["P2105", "P3005"],
    "VF6P": ["P2106", "P3006"],
    "VF2FP": ["P4803"],
    "VF5FP": ["P4804"],
    "VF6FP": ["P4805"],
}

# Apply conditions for 2019
for var, columns in vs_mapping_2019.items():
    df_2019_subset[var] = df_2019[columns].eq("Sí").any(axis=1)

# Apply conditions for 2015
for var, columns in vs_mapping_2015.items():
    df_2015_subset[var] = df_2015[columns].eq("Sí").any(axis=1)

In [ ]:
combined_df = pd.concat([df_2019_subset, df_2015_subset], ignore_index=True)
combined_df

,CUES,CCAA,PROV,ESTUDIO,AÑO,MES,VS1P,VS2P,VS3P,VS1FP,VF1P,VF2P,VF3P,VF4P,VF5P,VF6P,VF2FP,VF5FP,VF6FP
0,1,Andalucía,Granada,3235,2019,Septiembre,False,False,False,False,False,False,False,False,False,False,False,False,False
1,2,Andalucía,Granada,3235,2019,Septiembre,False,False,False,False,False,False,False,False,False,False,False,False,False
2,3,Andalucía,Granada,3235,2019,Septiembre,False,False,False,False,False,False,False,False,False,False,False,False,False
3,4,Andalucía,Granada,3235,2019,Septiembre,False,False,False,False,False,False,False,False,False,False,False,False,False
4,5,Andalucía,Granada,3235,2019,Septiembre,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19734,10254.0,Melilla (Ciudad Autónoma de),Melilla,3027,2014,Septiembre,False,False,False,False,False,False,False,False,False,False,False,False,False
19735,10255.0,Melilla (Ciudad Autónoma de),Melilla,3027,2014,Septiembre,False,False,False,False,False,False,False,False,False,False,False,False,False
19736,10256.0,Melilla (Ciudad Autónoma de),Melilla,3027,2014,Septiembre,False,False,False,False,False,False,False,False,False,False,False,False,False
19737,10257.0,Melilla (Ciudad Autónoma de),Melilla,3027,2014,Septiembre,False,False,False,False,False,False,False,False,False,False,False,False,False


# 3. Save as .csv

In [114]:
output_path = os.path.join(root_path, "datasets", "preliminary_merge", "PRE006-Macroencuestas2011Y2015.csv")
combined_df.to_csv(output_path, index=True)